# Python implementation of sounding analysis

In this exercise, you will repeat the sounding analysis from Week 2 using a Jupyter notebook

The first step is to load some packages. Don't worry too much about these at the moment. If you're using the Jupyter hub, everything is installed for you, and you can ignore the next instruction.

If you decide to install Jupyter notebooks locally on your computer, you can install the packages from within a cell as follows:

import sys
!{sys.executable} -m pip install [package name]

Note that pandas, numpy, datetime and matplotlib will probably be automatically installed with jupyter notebooks, but you will probably need to install metpy.


<b> Just run these next two cells with the triangle 'play' button in the menu </b>

In [ ]:
import pandas as pd
import numpy as np
import datetime as datetime
from matplotlib import pyplot as plt

In [ ]:
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units
import metpy.constants as mpconst

In [ ]:
# Change default to be better for skew-T
plt.rcParams['figure.figsize'] = (9, 7)

<b> Now we're going to load the sounding data. Remember to copy the file 'sounding_data.csv' into the same directory as this notebook (in your local directory, NOT in the workbooks directory). </b>

In [ ]:
SoundingData = pd.read_csv('Sounding_20230202_12Z.csv', sep=',', skiprows=4,
names = ['pressure', 'height', 'temperature', 'dewpoint', 'rh', 'w'], engine='python')
SoundingData.head()

In [ ]:
# Assign units to your data

hgt = SoundingData['height'].values * units.m
p = SoundingData['pressure'].values * units.hPa
T = SoundingData['temperature'].values * units.degC
Td = SoundingData['dewpoint'].values * units.degC

In [ ]:
# Let's have a look to see if it's loaded properly: 

print(SoundingData)

In [ ]:
fig = plt.figure(figsize=(9, 12))

# Set up a skewT axis
skew = SkewT(fig=fig, aspect = 100)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(SoundingData.pressure, SoundingData.temperature, 'r')
skew.plot(SoundingData.pressure, SoundingData.dewpoint, 'k')
# Add the relevant special lines
pressure_mixing_lines = np.arange(1000,300,100)* units.hPa
w = np.array([0.000125,0.00025,0.0005,0.001,0.002,0.004,0.008,0.016])[:, None] * units('g/g')

# Change the colours here if you prefer: (see https://matplotlib.org/stable/gallery/color/named_colors.html)
skew.plot_dry_adiabats(color='g', linewidth=1, t0=np.arange(-30+273.15, 60+273.15, 5) * units.K)
skew.plot_moist_adiabats(color='r', linewidth=1, t0=np.arange(-30+273.15, 60+273.15, 5) * units.K)
ml = skew.plot_mixing_lines(w,color='cadetblue', linewidth=1, pressure=p) #, pressure = pressure_mixing_lines)
skew.ax.set_ylim(1000, 300)
# Add xticks at every 1 degree.
xtick_locations = np.arange(-60, 40, 1)
skew.ax.set_xticks(xtick_locations)
# Only want to label every with tick
xtick_labels = [''] * len(xtick_locations)
xtick_labels[::5] = xtick_locations[::5].astype(str)
skew.ax.set_xticklabels(xtick_labels)
skew.ax.set_xlim(-40, 40)

# Label the mixing ratio isopleths
for val in w.flatten()[::1]:
    top_p = 600 * units.hPa
    dewpt = mpcalc.dewpoint(mpcalc.vapor_pressure(top_p, val))
    skew.ax.text(dewpt, top_p, str(val.to('g/kg').m),
                 horizontalalignment='center')

# Add the MetPy logo (just for fun)
add_metpy_logo(fig, 40, 35)

# Let's find the LCL pressure and temperature, and add them to the plot as a black dot:
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])
lcl_mixing_ratio = mpcalc.saturation_mixing_ratio(lcl_pressure, lcl_temperature) * 1000 # g/kg

# Print out the properties of the surface parcel at the LCL
print(f'The LCL pressure is {lcl_pressure:.1f}')
print(f'The LCL temperature is {lcl_temperature:.1f}')
print(f'The LCL mixing ratio is {lcl_mixing_ratio:.1f}')

# Mark the LCL on the skewT plot
skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

# Calculate full parcel profile and add to plot as black line
prof_pressures = np.arange(1000,200,-50) * units.hPa

prof = mpcalc.parcel_profile(prof_pressures, T[0], Td[0]).to('degC')

skew.plot(prof_pressures, prof, 'k', linewidth=2)
#skew.plot(prof_pressures, prof1, 'k', linewidth=3)

skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

In [ ]:
# We're going to find the closest height level to the 2000m, which is the top of the mountain.
mountain_top = 800 * units.hPa

closest_pressure = min(prof_pressures, key=lambda x:abs(x-mountain_top))
mountain_top_arg = np.argwhere(prof_pressures == closest_pressure)

print(mountain_top_arg, prof_pressures[mountain_top_arg])

We've just found out the index of the 800 hPa pressure level in the profile. Now we can bring the parcel back to the surface, just like we did manually last week. 

In [ ]:
# The saturation vapour pressure. This is the same as the vapour pressure 
# at this point, because the parcel is still saturated.

es_top = mpcalc.saturation_vapor_pressure(prof[mountain_top_arg]) # Pa
print(f'The vapour pressure at the mountain top is {es_top:.1f}')


In [ ]:
# Convert to mixing ratio. This will be conserved when the parcel returns to the surface
w = mpcalc.mixing_ratio(es_top, prof_pressures[mountain_top_arg]) * 1000 # g/kg
print(f'The mixing ratio after descent is {w:.1f}')

In [ ]:
# Now we need the saturation mixing ratio. To find this, we need to bring the parcel adiabatically 
# to the surface. This will conserve the potential temperature, so let's just find the potential
# temperature!

TK = prof[mountain_top_arg].to('degK')
P0 = 1000*units.hPa

potential_temperature_top = (TK * np.power((P0 / prof_pressures[mountain_top_arg]),0.286)).to('degC') 

print(f'The potential temperature at the mountain top is {potential_temperature_top:.1f}')

In [ ]:
# The potential temperature at 1000 hPa is the same as the potential temperature at the 
# mountain top. And at 1000hPa, the potential temperature is the same as the temperature.

temperature_at_1000hPa = potential_temperature_top

es = mpcalc.saturation_vapor_pressure(temperature_at_1000hPa) # Pa
print(f'The saturation vapour pressure after descent is {es:.1f}')

In [ ]:
# Saturation mixing ratio at 1000 hPa

ws = mpcalc.mixing_ratio(es, 1000 * units.hPa) * 1000 # g/kg
print(f'The mixing ratio after descent is {ws:.1f}')

In [ ]:
# Relative humidity: You do this bit!!!



# CHALLENGE #

Use the hydrostatic equation to check the height calculations in the sounding file. 

$ \frac{\partial P}{\partial z} = -\rho g $

Rewrite this as:

$ \Delta z = \frac{-1}{\rho g} \Delta P $

You'll need to start by calculating $\rho$

Then calculate $\Delta z$ for each pair of pressures, and sequentally add to the previous height (ie. use a for loop)

Hints: 
1. Use the first height from the sounding as your h[0]
2. For a more accurate result, calculate $\rho$ in the middle of each layer (ie. the average of the upper and lower values


Finally, make a plot of your height calculation against the one in the sounding file. Are they the same? 